In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
from scipy.stats import weibull_min
!pip install pulp
!pip install ortoolpy
from pulp import LpProblem,LpVariable, LpMinimize,LpMaximize, LpContinuous, lpSum, lpDot, value
from ortoolpy import model_min, addvars, addvals

# Define material budget model

In [ ]:
def material_budget_model(share_export_pri,
                          share_export_sec,
                          forming_yield_pri,
                          forming_yield_sec,
                          share_long_pri,
                          share_flat_pri,
                          share_tube_pri,
                          share_alloy_pri,
                          share_long_sec,
                          share_flat_sec,
                          share_tube_sec,
                          share_alloy_sec,
                          share_export_long,
                          share_export_flat,
                          share_export_tube,
                          share_export_alloy,
                          fabrication_yield_long,
                          fabrication_yield_flat,
                          fabrication_yield_tube,
                          fabrication_yield_alloy,
                          matrix_long_BU,
                          matrix_flat_BU,
                          matrix_tube_BU,
                          matrix_alloy_BU,
                          matrix_long_IF,
                          matrix_flat_IF,
                          matrix_tube_IF,
                          matrix_alloy_IF,
                          matrix_long_MM,
                          matrix_flat_MM,
                          matrix_tube_MM,
                          matrix_alloy_MM,
                          matrix_long_EE,
                          matrix_flat_EE,
                          matrix_tube_EE,
                          matrix_alloy_EE,
                          matrix_long_AU,
                          matrix_flat_AU,
                          matrix_tube_AU,
                          matrix_alloy_AU,
                          matrix_long_OT,
                          matrix_flat_OT,
                          matrix_tube_OT,
                          matrix_alloy_OT,
                          matrix_long_MP,
                          matrix_flat_MP,
                          matrix_tube_MP,
                          matrix_alloy_MP,
                          share_export_BU,
                          share_export_IF,
                          share_export_MM,
                          share_export_EE,
                          share_export_AU,
                          share_export_OT,
                          share_export_MP,
                          shape_BU,
                          scale_BU,
                          shape_IF,
                          scale_IF,
                          shape_MM,
                          scale_MM,
                          shape_EE,
                          scale_EE,
                          shape_AU,
                          scale_AU,
                          shape_OT,
                          scale_OT,
                          shape_MP,
                          scale_MP,
                          hiber_BU,
                          hiber_IF,
                          hiber_MM,
                          hiber_EE,
                          hiber_AU,
                          hiber_OT,
                          hiber_MP,
                          domestic_rec,
                          scrap_prep_eol,
                          scrap_prep_fabrication,
                          scrap_prep_forming,
                          sec_yield,
                          EI_pri,
                          EI_sec,
                          EI_hyd,
                          budget,
                          hyd_cap,
                          course50,
                          year,
                          scenario_index):
    
    Data['Var_pri'] = addvars(len(Data))
    Data['Var_sec'] = addvars(len(Data))
    Data['Var_hyd'] = addvars(len(Data))

    
    # net-exports of ingots and semis
    export_pri = Data.Var_pri * share_export_pri
    export_sec = Data.Var_sec * share_export_sec
    export_hyd = Data.Var_hyd * share_export_pri
    
    # forming scrap
    forming_pri = (Data.Var_pri - export_pri) * (1-forming_yield_pri)
    forming_sec = (Data.Var_sec - export_sec) * (1-forming_yield_sec)
    forming_hyd = (Data.Var_hyd - export_hyd) * (1-forming_yield_pri)
    
    # domestic production of finished steel
    pri_total = Data.Var_pri - export_pri - forming_pri
    pri_long  = pri_total * share_long_pri
    pri_flat  = pri_total * share_flat_pri
    pri_tube  = pri_total * share_tube_pri
    pri_alloy = pri_total * share_alloy_pri
    
    sec_total = Data.Var_sec - export_sec - forming_sec
    sec_long  = sec_total * share_long_sec
    sec_flat  = sec_total * share_flat_sec
    sec_tube  = sec_total * share_tube_sec
    sec_alloy = sec_total * share_alloy_sec
    
    hyd_total = Data.Var_hyd - export_hyd - forming_hyd
    hyd_long  = hyd_total * share_long_pri
    hyd_flat  = hyd_total * share_flat_pri
    hyd_tube  = hyd_total * share_tube_pri
    hyd_alloy = hyd_total * share_alloy_pri
    
    pro_long  = pri_long  + sec_long  + hyd_long
    pro_flat  = pri_flat  + sec_flat  + hyd_flat
    pro_tube  = pri_tube  + sec_tube  + hyd_tube
    pro_alloy = pri_alloy + sec_alloy + hyd_alloy
    
    # net-exports of finished steel
    export_long  = pro_long  * share_export_long
    export_flat  = pro_flat  * share_export_flat
    export_tube  = pro_tube  * share_export_tube
    export_alloy = pro_alloy * share_export_alloy
    
    # fabrication yield
    fabrication_long  = (pro_long  - export_long)  * (1-fabrication_yield_long)
    fabrication_flat  = (pro_flat  - export_flat)  * (1-fabrication_yield_flat)
    fabrication_tube  = (pro_tube  - export_tube)  * (1-fabrication_yield_tube)
    fabrication_alloy = (pro_alloy - export_alloy) * (1-fabrication_yield_alloy)
    
    # domestic use of finished steel
    use_long  = pro_long  - export_long  - fabrication_long
    use_flat  = pro_flat  - export_flat  - fabrication_flat
    use_tube  = pro_tube  - export_tube  - fabrication_tube
    use_alloy = pro_alloy - export_alloy - fabrication_alloy
   
    # domestic production of end-use goods
    pro_BU_long  = use_long  * matrix_long_BU
    pro_BU_flat  = use_flat  * matrix_flat_BU
    pro_BU_tube  = use_tube  * matrix_tube_BU
    pro_BU_alloy = use_alloy * matrix_alloy_BU
    
    pro_IF_long  = use_long  * matrix_long_IF
    pro_IF_flat  = use_flat  * matrix_flat_IF
    pro_IF_tube  = use_tube  * matrix_tube_IF
    pro_IF_alloy = use_alloy * matrix_alloy_IF

    pro_MM_long  = use_long  * matrix_long_MM
    pro_MM_flat  = use_flat  * matrix_flat_MM
    pro_MM_tube  = use_tube  * matrix_tube_MM
    pro_MM_alloy = use_alloy * matrix_alloy_MM

    pro_EE_long  = use_long  * matrix_long_EE
    pro_EE_flat  = use_flat  * matrix_flat_EE
    pro_EE_tube  = use_tube  * matrix_tube_EE
    pro_EE_alloy = use_alloy * matrix_alloy_EE

    pro_AU_long  = use_long  * matrix_long_AU
    pro_AU_flat  = use_flat  * matrix_flat_AU
    pro_AU_tube  = use_tube  * matrix_tube_AU
    pro_AU_alloy = use_alloy * matrix_alloy_AU
    
    pro_OT_long  = use_long  * matrix_long_OT
    pro_OT_flat  = use_flat  * matrix_flat_OT
    pro_OT_tube  = use_tube  * matrix_tube_OT
    pro_OT_alloy = use_alloy * matrix_alloy_OT
    
    pro_MP_long  = use_long  * matrix_long_MP
    pro_MP_flat  = use_flat  * matrix_flat_MP
    pro_MP_tube  = use_tube  * matrix_tube_MP
    pro_MP_alloy = use_alloy * matrix_alloy_MP

    pro_BU = pro_BU_long + pro_BU_flat + pro_BU_tube + pro_BU_alloy
    pro_IF = pro_IF_long + pro_IF_flat + pro_IF_tube + pro_IF_alloy
    pro_MM = pro_MM_long + pro_MM_flat + pro_MM_tube + pro_MM_alloy
    pro_EE = pro_EE_long + pro_EE_flat + pro_EE_tube + pro_EE_alloy
    pro_AU = pro_AU_long + pro_AU_flat + pro_AU_tube + pro_AU_alloy
    pro_OT = pro_OT_long + pro_OT_flat + pro_OT_tube + pro_OT_alloy
    pro_MP = pro_MP_long + pro_MP_flat + pro_MP_tube + pro_MP_alloy

    # net-exports of end-use goods
    export_BU = pro_BU * share_export_BU
    export_IF = pro_IF * share_export_IF
    export_MM = pro_MM * share_export_MM
    export_EE = pro_EE * share_export_EE
    export_AU = pro_AU * share_export_AU
    export_OT = pro_OT * share_export_OT
    export_MP = pro_MP * share_export_MP
    
    # inflow
    in_BU = pro_BU - export_BU
    in_IF = pro_IF - export_IF
    in_MM = pro_MM - export_MM
    in_EE = pro_EE - export_EE
    in_AU = pro_AU - export_AU
    in_OT = pro_OT - export_OT
    in_MP = pro_MP - export_MP

    year_complete = np.arange(1950,1951)
    ot_BU = np.repeat(0,len(year_complete))
    ot_IF = np.repeat(0,len(year_complete))
    ot_MM = np.repeat(0,len(year_complete))
    ot_EE = np.repeat(0,len(year_complete))
    ot_AU = np.repeat(0,len(year_complete))
    ot_OT = np.repeat(0,len(year_complete))
    ot_MP = np.repeat(0,len(year_complete))

    for k in range(1951,2051):
        
        # buildings
        ot_list_BU = (in_BU.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_BU.iloc[0:len(year_complete)],
                                        scale=scale_BU.iloc[0:len(year_complete)])))
        ot_sum_BU = sum(ot_list_BU)
        ot_BU = np.append(ot_BU,ot_sum_BU)
    
        # infrastructure
        ot_list_IF = (in_IF.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_IF.iloc[0:len(year_complete)],
                                        scale=scale_IF.iloc[0:len(year_complete)])))
        ot_sum_IF = sum(ot_list_IF)
        ot_IF = np.append(ot_IF,ot_sum_IF)
    
        # mechanical machinery
        ot_list_MM = (in_MM.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_MM.iloc[0:len(year_complete)],
                                        scale=scale_MM.iloc[0:len(year_complete)])))
        ot_sum_MM = sum(ot_list_MM)
        ot_MM = np.append(ot_MM,ot_sum_MM)
    
        # electrical equipment
        ot_list_EE = (in_EE.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_EE.iloc[0:len(year_complete)],
                                        scale=scale_EE.iloc[0:len(year_complete)])))
        ot_sum_EE = sum(ot_list_EE)
        ot_EE = np.append(ot_EE,ot_sum_EE)
    
        # automotive
        ot_list_AU = (in_AU.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_AU.iloc[0:len(year_complete)],
                                        scale=scale_AU.iloc[0:len(year_complete)])))
        ot_sum_AU = sum(ot_list_AU)
        ot_AU = np.append(ot_AU,ot_sum_AU)
    
        # other transport
        ot_list_OT = (in_OT.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_OT.iloc[0:len(year_complete)],
                                        scale=scale_OT.iloc[0:len(year_complete)])))
        ot_sum_OT = sum(ot_list_OT)
        ot_OT = np.append(ot_OT,ot_sum_OT)
    
        # metal products
        ot_list_MP = (in_MP.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_MP.iloc[0:len(year_complete)],
                                        scale=scale_MP.iloc[0:len(year_complete)])))
        ot_sum_MP = sum(ot_list_MP)
        ot_MP = np.append(ot_MP,ot_sum_MP)
        
        year_complete = np.append(year_complete,k)

    # end-of-life scrap
    EOL_BU = ot_BU * (1-hiber_BU)
    EOL_IF = ot_IF * (1-hiber_IF)
    EOL_MM = ot_MM * (1-hiber_MM)
    EOL_EE = ot_EE * (1-hiber_EE)
    EOL_AU = ot_AU * (1-hiber_AU)
    EOL_OT = ot_OT * (1-hiber_OT)
    EOL_MP = ot_MP * (1-hiber_MP)

    # scrap inputs to EAF
    EOL_sec= (EOL_BU + EOL_IF + EOL_MM + EOL_EE + EOL_AU + EOL_OT + EOL_MP) * domestic_rec
    fabrication_sec = fabrication_long + fabrication_flat + fabrication_tube + fabrication_alloy
    scrap_sec = (EOL_sec * scrap_prep_eol + 
                 fabrication_sec *  scrap_prep_fabrication + 
                 forming_sec * scrap_prep_forming) * sec_yield

    # CO2 emissions
    pro_pri = Data.Var_pri
    pro_sec = Data.Var_sec
    pro_hyd = Data.Var_hyd
    
    CO2 = (pro_pri * EI_pri) + (pro_sec * EI_sec) + (pro_hyd * EI_hyd)
    
    # inflow
    in_matrix = pd.DataFrame({0:in_BU,
                              1:in_IF,
                              2:in_MM,
                              3:in_EE,
                              4:in_AU,
                              5:in_OT,
                              6:in_MP,},)

    # outflow
    ot_matrix = pd.DataFrame({0:ot_BU,
                              1:ot_IF,
                              2:ot_MM,
                              3:ot_EE,
                              4:ot_AU,
                              5:ot_OT,
                              6:ot_MP,},)

    # stock
    NAS_matrix = in_matrix - ot_matrix
    st_matrix  = NAS_matrix.cumsum()
    st_sum     = st_matrix.sum(axis=1)

    model = LpProblem(sense=LpMaximize)
    model += lpSum(st_sum)
    
    for t in Data.index:
        
        # CO2 emission constraints
        model += CO2[t] <= budget[t]
    
        # scrap availability constraints
        model += Data.Var_sec[t] <= scrap_sec[t]
        
        # H2-DRI/EAF capacity constraints
        model += Data.Var_hyd[t] <= hyd_cap[t]
    
        # non-negative constraints
        model += Data.Var_pri[t] >=0
        model += Data.Var_sec[t] >=0
        model += Data.Var_hyd[t] >=0
        
    model.solve()
    Data['Val_pri'] = Data.Var_pri.apply(value)
    Data['Val_sec'] = Data.Var_sec.apply(value)
    Data['Val_hyd'] = Data.Var_hyd.apply(value)
    
    print('Objective function',value(model.objective))

########################################################################
# calculations with optimized numbers
########################################################################

    # net-exports of ingots and semis
    export_pri = Data.Val_pri * share_export_pri
    export_sec = Data.Val_sec * share_export_sec
    export_hyd = Data.Val_hyd * share_export_sec
    
    # forming scrap
    forming_pri = (Data.Val_pri - export_pri) * (1-forming_yield_pri)
    forming_sec = (Data.Val_sec - export_sec) * (1-forming_yield_sec)
    forming_hyd = (Data.Val_hyd - export_hyd) * (1-forming_yield_pri)
    
    # domestic production of finihsed steel
    pri_total = Data.Val_pri - export_pri - forming_pri
    pri_long  = pri_total * share_long_pri
    pri_flat  = pri_total * share_flat_pri
    pri_tube  = pri_total * share_tube_pri
    pri_alloy = pri_total * share_alloy_pri
    
    sec_total = Data.Val_sec - export_sec - forming_sec
    sec_long  = sec_total * share_long_sec
    sec_flat  = sec_total * share_flat_sec
    sec_tube  = sec_total * share_tube_sec
    sec_alloy = sec_total * share_alloy_sec
    
    hyd_total = Data.Val_hyd - export_hyd - forming_hyd
    hyd_long  = hyd_total * share_long_pri
    hyd_flat  = hyd_total * share_flat_pri
    hyd_tube  = hyd_total * share_tube_pri
    hyd_alloy = hyd_total * share_alloy_pri
    
    pro_long  = pri_long  + sec_long  + hyd_long
    pro_flat  = pri_flat  + sec_flat  + hyd_flat
    pro_tube  = pri_tube  + sec_tube  + hyd_tube
    pro_alloy = pri_alloy + sec_alloy + hyd_alloy
    
    # net-exports of finished steel
    export_long  = pro_long  * share_export_long
    export_flat  = pro_flat  * share_export_flat
    export_tube  = pro_tube  * share_export_tube
    export_alloy = pro_alloy * share_export_alloy
    
    # fabrication yield
    fabrication_long  = (pro_long  - export_long)  * (1-fabrication_yield_long)
    fabrication_flat  = (pro_flat  - export_flat)  * (1-fabrication_yield_flat)
    fabrication_tube  = (pro_tube  - export_tube)  * (1-fabrication_yield_tube)
    fabrication_alloy = (pro_alloy - export_alloy) * (1-fabrication_yield_alloy)
    
    # domestic use of finished steel
    use_long  = pro_long   - export_long  - fabrication_long
    use_flat  = pro_flat   - export_flat  - fabrication_flat
    use_tube  = pro_tube   - export_tube  - fabrication_tube
    use_alloy = pro_alloy  - export_alloy - fabrication_alloy
   
    # domestic production of end-use goods
    pro_BU_long  = use_long  * matrix_long_BU
    pro_BU_flat  = use_flat  * matrix_flat_BU
    pro_BU_tube  = use_tube  * matrix_tube_BU
    pro_BU_alloy = use_alloy * matrix_alloy_BU
    
    pro_IF_long  = use_long  * matrix_long_IF
    pro_IF_flat  = use_flat  * matrix_flat_IF
    pro_IF_tube  = use_tube  * matrix_tube_IF
    pro_IF_alloy = use_alloy * matrix_alloy_IF

    pro_MM_long  = use_long  * matrix_long_MM
    pro_MM_flat  = use_flat  * matrix_flat_MM
    pro_MM_tube  = use_tube  * matrix_tube_MM
    pro_MM_alloy = use_alloy * matrix_alloy_MM

    pro_EE_long  = use_long  * matrix_long_EE
    pro_EE_flat  = use_flat  * matrix_flat_EE
    pro_EE_tube  = use_tube  * matrix_tube_EE
    pro_EE_alloy = use_alloy * matrix_alloy_EE

    pro_AU_long  = use_long  * matrix_long_AU
    pro_AU_flat  = use_flat  * matrix_flat_AU
    pro_AU_tube  = use_tube  * matrix_tube_AU
    pro_AU_alloy = use_alloy * matrix_alloy_AU
    
    pro_OT_long  = use_long  * matrix_long_OT
    pro_OT_flat  = use_flat  * matrix_flat_OT
    pro_OT_tube  = use_tube  * matrix_tube_OT
    pro_OT_alloy = use_alloy * matrix_alloy_OT
    
    pro_MP_long  = use_long  * matrix_long_MP
    pro_MP_flat  = use_flat  * matrix_flat_MP
    pro_MP_tube  = use_tube  * matrix_tube_MP
    pro_MP_alloy = use_alloy * matrix_alloy_MP

    pro_BU = pro_BU_long + pro_BU_flat + pro_BU_tube + pro_BU_alloy
    pro_IF = pro_IF_long + pro_IF_flat + pro_IF_tube + pro_IF_alloy
    pro_MM = pro_MM_long + pro_MM_flat + pro_MM_tube + pro_MM_alloy
    pro_EE = pro_EE_long + pro_EE_flat + pro_EE_tube + pro_EE_alloy
    pro_AU = pro_AU_long + pro_AU_flat + pro_AU_tube + pro_AU_alloy
    pro_OT = pro_OT_long + pro_OT_flat + pro_OT_tube + pro_OT_alloy
    pro_MP = pro_MP_long + pro_MP_flat + pro_MP_tube + pro_MP_alloy

    # net-exports of end-use goods
    export_BU = pro_BU * share_export_BU
    export_IF = pro_IF * share_export_IF
    export_MM = pro_MM * share_export_MM
    export_EE = pro_EE * share_export_EE
    export_AU = pro_AU * share_export_AU
    export_OT = pro_OT * share_export_OT
    export_MP = pro_MP * share_export_MP
    
    # inflow
    in_BU = pro_BU - export_BU
    in_IF = pro_IF - export_IF
    in_MM = pro_MM - export_MM
    in_EE = pro_EE - export_EE
    in_AU = pro_AU - export_AU
    in_OT = pro_OT - export_OT
    in_MP = pro_MP - export_MP

    year_complete = np.arange(1950,1951)
    ot_BU = np.repeat(0,len(year_complete))
    ot_IF = np.repeat(0,len(year_complete))
    ot_MM = np.repeat(0,len(year_complete))
    ot_EE = np.repeat(0,len(year_complete))
    ot_AU = np.repeat(0,len(year_complete))
    ot_OT = np.repeat(0,len(year_complete))
    ot_MP = np.repeat(0,len(year_complete))

    for k in range(1951,2051):
        
        # buildings
        ot_list_BU = (in_BU.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_BU.iloc[0:len(year_complete)],
                                        scale=scale_BU.iloc[0:len(year_complete)])))
        ot_sum_BU = sum(ot_list_BU)
        ot_BU = np.append(ot_BU,ot_sum_BU)
    
        # infrastructure
        ot_list_IF = (in_IF.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_IF.iloc[0:len(year_complete)],
                                        scale=scale_IF.iloc[0:len(year_complete)])))
        ot_sum_IF = sum(ot_list_IF)
        ot_IF = np.append(ot_IF,ot_sum_IF)
    
        # mechanical machinery
        ot_list_MM = (in_MM.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_MM.iloc[0:len(year_complete)],
                                        scale=scale_MM.iloc[0:len(year_complete)])))
        ot_sum_MM = sum(ot_list_MM)
        ot_MM = np.append(ot_MM,ot_sum_MM)
    
        # electrical equipment
        ot_list_EE = (in_EE.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_EE.iloc[0:len(year_complete)],
                                        scale=scale_EE.iloc[0:len(year_complete)])))
        ot_sum_EE = sum(ot_list_EE)
        ot_EE = np.append(ot_EE,ot_sum_EE)
    
        # automotive
        ot_list_AU = (in_AU.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_AU.iloc[0:len(year_complete)],
                                        scale=scale_AU.iloc[0:len(year_complete)])))
        ot_sum_AU = sum(ot_list_AU)
        ot_AU = np.append(ot_AU,ot_sum_AU)
    
        # other transport
        ot_list_OT = (in_OT.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_OT.iloc[0:len(year_complete)],
                                        scale=scale_OT.iloc[0:len(year_complete)])))
        ot_sum_OT = sum(ot_list_OT)
        ot_OT = np.append(ot_OT,ot_sum_OT)
    
        # metal products
        ot_list_MP = (in_MP.iloc[0:len(year_complete)] 
                      *(weibull_min.pdf(k-year_complete,
                                        c=shape_MP.iloc[0:len(year_complete)],
                                        scale=scale_MP.iloc[0:len(year_complete)])))
        ot_sum_MP = sum(ot_list_MP)
        ot_MP = np.append(ot_MP,ot_sum_MP)
        
        year_complete = np.append(year_complete,k)

    # end-of-life scrap
    EOL_BU = ot_BU * (1-hiber_BU)
    EOL_IF = ot_IF * (1-hiber_IF)
    EOL_MM = ot_MM * (1-hiber_MM)
    EOL_EE = ot_EE * (1-hiber_EE)
    EOL_AU = ot_AU * (1-hiber_AU)
    EOL_OT = ot_OT * (1-hiber_OT)
    EOL_MP = ot_MP * (1-hiber_MP)

    # scrap inputs to EAF
    EOL_sec= (EOL_BU + EOL_IF + EOL_MM + EOL_EE + EOL_AU + EOL_OT + EOL_MP) * domestic_rec
    fabrication_sec = fabrication_long + fabrication_flat + fabrication_tube + fabrication_alloy
    scrap_sec = (EOL_sec * scrap_prep_eol + 
                 fabrication_sec *  scrap_prep_fabrication + 
                 forming_sec * scrap_prep_forming) * sec_yield
    
    scrap = pd.DataFrame({'EOL':EOL_sec * scrap_prep_eol,
                          'Fabrication':fabrication_sec *  scrap_prep_fabrication,
                          'Forming':forming_sec * scrap_prep_forming},)
    
    # CO2 emissions
    pro_pri = Data.Val_pri
    pro_sec = Data.Val_sec
    pro_hyd = Data.Val_hyd
    
    pro_couse50 = pro_pri * course50
    pro_BF = pro_pri - pro_couse50
    
    production = pd.DataFrame({'BF/BOF'   :pro_BF,
                               'H$_2$-BF/BOF+CCS':pro_couse50,
                               'Scrap-EAF':pro_sec,
                               'H$_2$-DRI/EAF':pro_hyd},)
    
    CO2 = pd.DataFrame({'BF/BOF'   :pro_pri * EI_pri,
                        'Scrap-EAF':pro_sec * EI_sec,
                        'H$_2$-DRI/EAF':pro_hyd * EI_hyd,},)
    
    # domestic production of finished steel
    steel_matrix = pd.DataFrame({'Carbon steel (long)' :pro_long,
                                 'Carbon steel (flat)' :pro_flat,
                                 'Carbon steel (tube)' :pro_tube,
                                 'Alloy steel'         :pro_alloy,},)
    
    # domestic production of end-use goods
    goods_matrix = pd.DataFrame({'Buildings'           :pro_BU,
                                 'Infrastructure'      :pro_IF,
                                 'Mechanical machinery':pro_MM,
                                 'Electrical equipment':pro_EE,
                                 'Automobiles'         :pro_AU,
                                 'Other transport'     :pro_OT,
                                 'Metal products'      :pro_MP,},)
    
    # inflow
    in_matrix = pd.DataFrame({'Buildings'           :in_BU,
                              'Infrastructure'      :in_IF,
                              'Mechanical machinery':in_MM,
                              'Electrical equipment':in_EE,
                              'Automobiles'         :in_AU,
                              'Other transport'     :in_OT,
                              'Metal products'      :in_MP,},)

    # outflow
    ot_matrix = pd.DataFrame({'Buildings'           :ot_BU,
                              'Infrastructure'      :ot_IF,
                              'Mechanical machinery':ot_MM,
                              'Electrical equipment':ot_EE,
                              'Automobiles'         :ot_AU,
                              'Other transport'     :ot_OT,
                              'Metal products'      :ot_MP,},)

    # stock
    NAS_matrix = in_matrix - ot_matrix
    st_matrix  = NAS_matrix.cumsum()
    
    production   = production.rename(index = year)
    steel_matrix = steel_matrix.rename(index = year)
    goods_matrix = goods_matrix.rename(index = year)
    CO2          = CO2.rename(index = year)
    in_matrix    = in_matrix.rename(index = year)
    ot_matrix    = ot_matrix.rename(index = year)
    st_matrix    = st_matrix.rename(index = year)
    
    scenario_index = scenario_index
    
    with pd.ExcelWriter('Results'+'/Scenario_'+str(scenario_index) + '.xlsx') as writer:
        production.to_excel(writer, sheet_name='crude_steel')
        steel_matrix.to_excel(writer, sheet_name='pro_steel')
        goods_matrix.to_excel(writer, sheet_name='pro_goods')
        CO2.to_excel(writer, sheet_name='CO2')
        in_matrix.to_excel(writer, sheet_name='inflow')
        ot_matrix.to_excel(writer, sheet_name='outflow')
        st_matrix.to_excel(writer, sheet_name='stock')
        scrap.to_excel(writer, sheet_name='scrap')

# Import data

In [ ]:
matrix_long  = pd.read_excel (io = r'data_inputs.xlsx', sheet_name="matrix_long")
matrix_flat  = pd.read_excel (io = r'data_inputs.xlsx', sheet_name="matrix_flat")
matrix_tube  = pd.read_excel (io = r'data_inputs.xlsx', sheet_name="matrix_tube")
matrix_alloy = pd.read_excel (io = r'data_inputs.xlsx', sheet_name="matrix_alloy")

# Run the model

In [ ]:
strategy = ['NO','ENE','H-BF','H-DRI','REC','FAB','LTE']

for sheet in strategy:
    sheet_name = sheet
    Data = pd.read_excel (io = r'data_inputs.xlsx', sheet_name = sheet_name,nrows=101)
    
    material_budget_model(share_export_pri        = Data.share_export_pri,
                          share_export_sec        = Data.share_export_sec,
                          forming_yield_pri       = Data.forming_yield_pri,
                          forming_yield_sec       = Data.forming_yield_sec,
                          share_long_pri          = Data.share_long_pri,
                          share_flat_pri          = Data.share_flat_pri,
                          share_tube_pri          = Data.share_tube_pri,
                          share_alloy_pri         = Data.share_alloy_pri,
                          share_long_sec          = Data.share_long_sec,
                          share_flat_sec          = Data.share_flat_sec,
                          share_tube_sec          = Data.share_tube_sec,
                          share_alloy_sec         = Data.share_alloy_sec,
                          share_export_long       = Data.share_export_long,
                          share_export_flat       = Data.share_export_flat,
                          share_export_tube       = Data.share_export_tube,
                          share_export_alloy      = Data.share_export_alloy,
                          fabrication_yield_long  = Data.fabrication_yield_long,
                          fabrication_yield_flat  = Data.fabrication_yield_flat,
                          fabrication_yield_tube  = Data.fabrication_yield_tube,
                          fabrication_yield_alloy = Data.fabrication_yield_alloy,
                          matrix_long_BU          = matrix_long.BU,
                          matrix_flat_BU          = matrix_flat.BU,
                          matrix_tube_BU          = matrix_tube.BU,
                          matrix_alloy_BU         = matrix_alloy.BU,
                          matrix_long_IF          = matrix_long.IF,
                          matrix_flat_IF          = matrix_flat.IF,
                          matrix_tube_IF          = matrix_tube.IF,
                          matrix_alloy_IF         = matrix_alloy.IF,
                          matrix_long_MM          = matrix_long.MM,
                          matrix_flat_MM          = matrix_flat.MM,
                          matrix_tube_MM          = matrix_tube.MM,
                          matrix_alloy_MM         = matrix_alloy.MM,
                          matrix_long_EE          = matrix_long.EE,
                          matrix_flat_EE          = matrix_flat.EE,
                          matrix_tube_EE          = matrix_tube.EE,
                          matrix_alloy_EE         = matrix_alloy.EE,  
                          matrix_long_AU          = matrix_long.AU,
                          matrix_flat_AU          = matrix_flat.AU,
                          matrix_tube_AU          = matrix_tube.AU,
                          matrix_alloy_AU         = matrix_alloy.AU,
                          matrix_long_OT          = matrix_long.OT,
                          matrix_flat_OT          = matrix_flat.OT,
                          matrix_tube_OT          = matrix_tube.OT,
                          matrix_alloy_OT         = matrix_alloy.OT,
                          matrix_long_MP          = matrix_long.MP,
                          matrix_flat_MP          = matrix_flat.MP,
                          matrix_tube_MP          = matrix_tube.MP,
                          matrix_alloy_MP         = matrix_alloy.MP, 
                          share_export_BU         = Data.share_export_BU,
                          share_export_IF         = Data.share_export_IF,
                          share_export_MM         = Data.share_export_MM,
                          share_export_EE         = Data.share_export_EE,
                          share_export_AU         = Data.share_export_AU,
                          share_export_OT         = Data.share_export_OT,
                          share_export_MP         = Data.share_export_MP,
                          shape_BU                = Data.shape_BU,
                          scale_BU                = Data.scale_BU,
                          shape_IF                = Data.shape_IF,
                          scale_IF                = Data.scale_IF,
                          shape_MM                = Data.shape_MM,
                          scale_MM                = Data.scale_MM,
                          shape_EE                = Data.shape_EE,
                          scale_EE                = Data.scale_EE,
                          shape_AU                = Data.shape_AU,
                          scale_AU                = Data.scale_AU,
                          shape_OT                = Data.shape_OT,
                          scale_OT                = Data.scale_OT,
                          shape_MP                = Data.shape_MP,
                          scale_MP                = Data.scale_MP,
                          hiber_BU                = Data.hiber_BU,
                          hiber_IF                = Data.hiber_IF,
                          hiber_MM                = Data.hiber_MM,
                          hiber_EE                = Data.hiber_EE,
                          hiber_AU                = Data.hiber_AU,
                          hiber_OT                = Data.hiber_OT,
                          hiber_MP                = Data.hiber_MP,
                          domestic_rec            = Data.domestic_rec,
                          scrap_prep_eol          = Data.scrap_prep_eol,
                          scrap_prep_fabrication  = Data.scrap_prep_fabrication,
                          scrap_prep_forming      = Data.scrap_prep_forming,
                          sec_yield               = Data.sec_yield,
                          EI_pri                  = Data.EI_pri,
                          EI_sec                  = Data.EI_sec,
                          EI_hyd                  = Data.EI_hyd,
                          budget                  = Data.budget,
                          hyd_cap                 = Data.hyd_cap,
                          course50                = Data.course50,
                          year                    = Data.year,
                          scenario_index          = sheet_name)